<a href="https://colab.research.google.com/github/yojulab/learn_deeplearning/blob/master/codes/Regressions/Regression_simpleLinear_ozone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 오존 측정치(ozone.csv) : feature - 태양세기, 바람세기, 온도("Solar.R","Wind","Temp") / label - 오존치("Ozone")

###1개 독립변수와 종속변수 사용

In [ ]:
import tensorflow as tf
tf.__version__

'2.9.2'

###Raw Data Loading

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./ozone.csv')
# df
# df.describe()
# df.info()
df.keys()

Index(['Ozone', 'Solar.R', 'Wind', 'Temp', 'Month', 'Day'], dtype='object')

In [ ]:
train_df = df[['Temp','Ozone']]
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Temp    153 non-null    int64  
 1   Ozone   116 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 2.5 KB


###Data PreProcessing

####Missing Value(결측치)

In [ ]:
#Remove Row
train_preprocessing = train_df.dropna(how='any')
train_preprocessing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 152
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Temp    116 non-null    int64  
 1   Ozone   116 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 2.7 KB


####이상치 처리 : 생략

####정규화(Normalization)

#####각 columns 분리

In [ ]:
feature_normalization = train_preprocessing['Temp'].values.reshape(-1,1) # 2차원 Numpy 변환
feature_normalization.shape

(116, 1)

In [ ]:
target_normalization = train_preprocessing['Ozone'].values.reshape(-1,1) # 2차원 Numpy 변환
target_normalization.shape

(116, 1)

#####Min-Max Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
feature_minmaxscaler = MinMaxScaler()
feature_minmaxscaler.fit(feature_normalization)
feature_minmaxscaler.scale_, feature_minmaxscaler.min_

(array([0.025]), array([-1.425]))

In [ ]:
feature_set = feature_minmaxscaler.transform(feature_normalization)
# feature_set

In [ ]:
target_minmaxscaler = MinMaxScaler()
target_minmaxscaler.fit(target_normalization)
target_minmaxscaler.min_

array([-0.00598802])

In [ ]:
target_set = target_minmaxscaler.transform(target_normalization)
# target_set

###Model 구현

In [ ]:
import tensorflow as tf

#### input layer

In [ ]:
input = tf.keras.layers.Input(shape=(1,))

####hidden layers

In [ ]:
hiddens = tf.keras.layers.Flatten()(input)

#### output layer

In [ ]:
output = tf.keras.layers.Dense(units=1, activation='linear')(hiddens)

#### Model 

In [ ]:
model = tf.keras.Model(inputs=input, outputs=output)

###Model 설정

####Optimizer 선택

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

####Loss function 선택

In [ ]:
loss = tf.keras.losses.MeanSquaredError()
#loss = tf.keras.losses.MeanAbsoluteError()

In [ ]:
model.compile(optimizer=optimizer, loss=loss)

###Model 학습

In [ ]:
# rename
feature = feature_set.copy()
target = target_set.copy()

In [ ]:
epoch = 500
history = model.fit(x=feature, y=target, epochs=epoch, shuffle=True, verbose=0)

###Model예측

####예측할 데이터 생성

In [ ]:
#온도 화씨 70도 일 경우
import numpy as np

unkown_data = np.array([[70]])

####PreProcessing 적용

In [ ]:
unkown_data_minmaxscaler = feature_minmaxscaler.transform(unkown_data)

In [ ]:
predict_result = model.predict(unkown_data_minmaxscaler)
predict_result

1/1 [==============================] - 0s 22ms/step


array([[0.13868183]], dtype=float32)

####PreProcessing 역적용

In [ ]:
target_minmaxscaler.inverse_transform(predict_result)

array([[24.159864]], dtype=float32)